В качестве домашнего задания вам предлагается поработать над предсказанием погоды. Файл с данными вы найдете в соответствующей директории. Вам будет доступен датасет weather.csv, ПЕРВЫЕ 75% (shuffle = False) которого нужно взять для обучения, последние 25% - для тестирования.

Требуется построить 4 модели которые будут предсказывать целевую переменную <b>RainTomorrow</b> с помощью:

   1. логистической регрессии [sklearn.linear_model.LogisticRegression](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression)
   
   2. метода ближайших соседей [sklearn.neighbors](https://scikit-learn.org/stable/modules/neighbors.html)

   3. Байесовского классификатора [sklearn.naive_bayes](https://scikit-learn.org/stable/modules/naive_bayes.html)
   
   4. логистической регрессии реализованной самостоятельно

Затем следует сравнить результаты моделей (по качеству и времени выполнения) и сделать вывод о том, какая модель и с какими параметрами даёт лучшие результаты.

Не забывайте о том, что работа с признаками играет очень большую роль в построении хорошей модели.

Краткое описание данных:

    Date - Дата наблюдений
    Location - Название локации, в которой расположена метеорологическая станция
    MinTemp - Минимальная температура в градусах цельсия
    MaxTemp - Максимальная температура в градусах цельсия
    Rainfall - Количество осадков, зафиксированных за день в мм
    Evaporation - Так называемое "pan evaporation" класса А (мм) за 24 часа до 9 утра
    Sunshine - Число солнечных часов за день
    WindGustDir - направление самого сильного порыва ветра за последние 24 часа
    WindGustSpeed - скорость (км / ч) самого сильного порыва ветра за последние 24 часа
    WindDir9am - направление ветра в 9 утра

In [118]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import numpy as np
import seaborn as sns
import time
from sklearn.datasets import make_classification
import matplotlib.pyplot as plt
%matplotlib notebook
from sklearn.feature_selection import f_classif
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold, ParameterGrid
from sklearn.metrics import accuracy_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV

In [127]:
X = pd.read_csv('weather.csv')
y = X.RainTomorrow.replace({'No':0, 'Yes': 1})
X.RainToday.replace({'No':0, 'Yes': 1})
del X['RainTomorrow']
del X['Unnamed: 0']

In [36]:
print(X)

              Date Location  MinTemp  MaxTemp  Rainfall  Evaporation  \
0       2008-12-01   Albury     13.4     22.9       0.6          NaN   
1       2008-12-02   Albury      7.4     25.1       0.0          NaN   
2       2008-12-03   Albury     12.9     25.7       0.0          NaN   
3       2008-12-04   Albury      9.2     28.0       0.0          NaN   
4       2008-12-05   Albury     17.5     32.3       1.0          NaN   
...            ...      ...      ...      ...       ...          ...   
142188  2017-06-20    Uluru      3.5     21.8       0.0          NaN   
142189  2017-06-21    Uluru      2.8     23.4       0.0          NaN   
142190  2017-06-22    Uluru      3.6     25.3       0.0          NaN   
142191  2017-06-23    Uluru      5.4     26.9       0.0          NaN   
142192  2017-06-24    Uluru      7.8     27.0       0.0          NaN   

        Sunshine WindGustDir  WindGustSpeed WindDir9am  ... WindSpeed3pm  \
0            NaN           W           44.0          W  ...

Процент пропусков в данных


In [38]:
X.isna().sum()/X.shape[0]

Date             0.000000
Location         0.000000
MinTemp          0.004480
MaxTemp          0.002265
Rainfall         0.009888
Evaporation      0.427890
Sunshine         0.476929
WindGustDir      0.065615
WindGustSpeed    0.065193
WindDir9am       0.070418
WindDir3pm       0.026570
WindSpeed9am     0.009480
WindSpeed3pm     0.018496
Humidity9am      0.012476
Humidity3pm      0.025388
Pressure9am      0.098556
Pressure3pm      0.098324
Cloud9am         0.377353
Cloud3pm         0.401525
Temp9am          0.006358
Temp3pm          0.019171
RainToday        0.009888
dtype: float64

Удаление столбцов в которых много пропусков
(больше 40%)

In [112]:
X= X.drop(columns=['Sunshine', 'Evaporation'])


Заполнение пропусков в данных


In [113]:
categorial = list(X.select_dtypes(include = ['object']).columns.values)
for col in categorial:
  if(X[col].isnull().values.any()):
    X[col].fillna(X[col].mode()[0],inplace=True)
numerical=list(X.select_dtypes(include = ['int64', 'float64']).columns.values)
for col in numerical:
  if(X[col].isnull().values.any()):
    X[col].fillna(X[col].median(),inplace=True)


Преобразование направлений ветра в градусы

In [114]:
dic={'N':0,'W':270,'E':90,'S':180,'SE':135,'SW':225,'NE':45,'NW':315,'NNE':22,'ENE':67, 'ESE':112,'SSE':157,'SSW':202,'WSW':247,'WNW':292,'NNW':337}
#X['WindGustDir']=X['WindGustDir'].apply(lambda x: dic[x])
X['WindGustDir']=[dic[x] for x in X['WindGustDir']]
X['WindDir9am']=[dic[x] for x in X['WindDir9am']]
X['WindDir3pm']=[dic[x] for x in X['WindDir3pm']]


Преобразование даты

In [115]:
#2008-12-01 --> 12
def encode(arr):
  return int(arr[5:7])
X['Date']=[encode(x) for x in X['Date']]


One-Hot Encoding для колонки Location(для логистической регрессии увеличил accuracy на 0.6%)

In [ ]:
encoder = OneHotEncoder(sparse_output=False)
categorical_columns=['Location']
# Apply one-hot encoding to the categorical columns
one_hot_encoded = encoder.fit_transform(np.array(X[categorical_columns]).reshape(-1,1))

#Create a DataFrame with the one-hot encoded columns
#We use get_feature_names_out() to get the column names for the encoded data
one_hot_df = pd.DataFrame(one_hot_encoded, columns=encoder.get_feature_names_out(categorical_columns))

# Concatenate the one-hot encoded dataframe with the original dataframe
X = pd.concat([X, one_hot_df], axis=1)

# Drop the original categorical columns
X = X.drop(categorical_columns, axis=1)
print(X)

Строим матрицу корелляции

In [116]:
cor=X.corr().round(2)
cor.style.background_gradient(cmap='coolwarm')

,Date,MinTemp,MaxTemp,Rainfall,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm
Date,1.000000,-0.200000,-0.160000,-0.030000,0.050000,0.060000,0.030000,0.040000,0.050000,0.060000,-0.090000,-0.020000,0.030000,0.030000,-0.010000,-0.000000,-0.140000,-0.170000
MinTemp,-0.200000,1.000000,0.730000,0.100000,-0.160000,0.170000,-0.030000,-0.180000,0.180000,0.170000,-0.230000,0.010000,-0.420000,-0.430000,0.040000,-0.000000,0.900000,0.700000
MaxTemp,-0.160000,0.730000,1.000000,-0.070000,-0.190000,0.070000,-0.190000,-0.150000,0.010000,0.050000,-0.500000,-0.500000,-0.310000,-0.400000,-0.230000,-0.220000,0.880000,0.970000
Rainfall,-0.030000,0.100000,-0.070000,1.000000,0.040000,0.130000,0.070000,0.030000,0.090000,0.060000,0.220000,0.250000,-0.160000,-0.120000,0.170000,0.140000,0.010000,-0.080000
WindGustDir,0.050000,-0.160000,-0.190000,0.040000,1.000000,0.140000,0.290000,0.530000,-0.020000,0.070000,0.060000,0.030000,-0.150000,-0.090000,0.060000,0.060000,-0.180000,-0.200000
WindGustSpeed,0.060000,0.170000,0.070000,0.130000,0.140000,1.000000,0.110000,0.140000,0.580000,0.660000,-0.210000,-0.030000,-0.430000,-0.380000,0.050000,0.070000,0.150000,0.030000
WindDir9am,0.030000,-0.030000,-0.190000,0.070000,0.290000,0.110000,1.000000,0.240000,0.140000,0.120000,0.050000,0.120000,-0.100000,-0.030000,0.050000,0.030000,-0.100000,-0.200000
WindDir3pm,0.040000,-0.180000,-0.150000,0.030000,0.530000,0.140000,0.240000,1.000000,0.020000,0.080000,0.020000,-0.050000,-0.160000,-0.100000,0.040000,0.050000,-0.180000,-0.150000
WindSpeed9am,0.050000,0.180000,0.010000,0.090000,-0.020000,0.580000,0.140000,0.020000,1.000000,0.510000,-0.270000,-0.030000,-0.210000,-0.160000,0.010000,0.030000,0.130000,0.010000
WindSpeed3pm,0.060000,0.170000,0.050000,0.060000,0.070000,0.660000,0.120000,0.080000,0.510000,1.000000,-0.140000,0.020000,-0.280000,-0.240000,0.030000,0.010000,0.160000,0.030000


Удаляем признаки с высокой корелляцией(более 85%)

In [117]:
X=X.drop(columns=['MaxTemp','MinTemp', 'Pressure9am'])

StandartScaller

In [122]:
numerical=list(X.select_dtypes(include = ['int64', 'float64']).columns.values)
scaler = StandardScaler().fit(X[numerical])
X[numerical]=scaler.transform(X[numerical])
#y=scaler.transform(y)


Ищем наиболее значимые признаки

In [79]:
# импортируем модуль stats из библиотеки scipy
from scipy import stats

# создадим два списка, один для названий признаков, второй для корреляций
columns, correlations = [], []

# пройдемся по всем столбцам датафрейма кроме целевой переменной
for col in X[numerical].columns:
  # поместим название признака в список columns
  columns.append(col)
  # рассчитаем корреляцию этого признака с целевой переменной
  # и поместим результат в список корреляций
  correlations.append(stats.pointbiserialr(X[col], y)[0])

# создадим датафрейм на основе заполненных списков
# и применим градиентную цветовую схему
pd.DataFrame({'column': columns, 'correlation': correlations}).style.background_gradient()


,column,correlation
0,Date,0.007328
1,Rainfall,0.235087
2,WindGustDir,0.062288
3,WindGustSpeed,0.224766
4,WindDir9am,0.042018
5,WindDir3pm,0.044234
6,WindSpeed9am,0.090446
7,WindSpeed3pm,0.086973
8,Humidity9am,0.255292
9,Humidity3pm,0.439741


Разделение на train and test

In [123]:
x_train, x_test, y_train, y_test = train_test_split(X[numerical], y, test_size=0.25, random_state=1)

Логистическая регрессия из библиотеки

In [124]:
clf = LogisticRegression(random_state=0).fit(x_train, y_train)
print(accuracy_score(clf.predict(x_test),y_test))


0.8409800556977692


In [126]:
print(classification_report(y_test, clf.predict(x_test)))

              precision    recall  f1-score   support

           0       0.86      0.94      0.90     27601
           1       0.71      0.48      0.58      7948

    accuracy                           0.84     35549
   macro avg       0.79      0.71      0.74     35549
weighted avg       0.83      0.84      0.83     35549



K Nearest Neighbours

In [125]:
knn = KNeighborsClassifier(n_neighbors=3)
knn.fit(x_train, y_train)
print(accuracy_score(knn.predict(x_test),y_test))


0.8275900869222763


In [ ]:
parameters = { 'n_neighbors':range(1, 8,2)}
knn = KNeighborsClassifier()
clf = GridSearchCV(knn, parameters)
clf.fit(x_train, y_train)

sorted(clf.cv_results_.keys())

PCA(хорошо использовать для KNN для увеличения скорости работы)

In [ ]:
pca = PCA(n_components=10)

Xpca=pca.fit_transform(X[numerical])
x_train, x_test, y_train, y_test = train_test_split(Xpca, y, test_size=0.25, random_state=1)
clf = KNeighborsClassifier(4).fit(x_train, y_train)
print(accuracy_score(clf.predict(x_test),y_test))

Байесоввский классификатор

In [77]:

gnb = GaussianNB().fit(x_train, y_train)
print(accuracy_score(gnb.predict(x_test),y_test))

0.8226954344707306


### Реализация логистической регрессии
__Логистическая регрессия__

$$p(y|x) = a(x, \theta) = \sigma(\langle x, \theta \rangle) = \frac{1}{1 + \exp(-\langle \theta, x_i \rangle)}$$

In [3]:
theta = np.array([1, 2, 3])

X =  np.array([[ 1,  1, 1],
               [-1, -2, 1],
               [-1, -2, 2],
               [-2, -2, -3]
              ])

y = np.array([1, 1, 0, 0])

In [6]:
def probability(theta, X):
    result=1/(1+np.exp(-np.dot(X,theta)))
    return result
prob = probability(theta, X)


assert type(prob) == np.ndarray, 'Возвращается неверный тип'
assert prob.shape == (X.shape[0],), 'Неверный размер массива'
assert (prob.round(3) == [0.998, 0.119, 0.731, 0.]).all(), 'Функция считается неверно'

Функция предсказания метки класса, получает на вход вероятности принадлежности к классу 1 и выдает метки классов $y \in \{0, 1\}$

In [7]:
def binary_class_prediction(theta, X, threshold =.5):
    prob =  probability(theta, X)
    result=np.where(prob>=0.5,1,0)
    return result

y_pred = binary_class_prediction(theta, X)


assert type(y_pred) == np.ndarray, 'Возвращается неверный тип'
assert y_pred.shape == (X.shape[0],), 'Неверный размер массива'
assert min(y_pred) == 0, 'Функция считается неверно'
assert max(y_pred) == 1, 'Функция считается неверно'

__Функционал качества логистической регрессии__

Запишем правдободовие выборки для меток класса $y \in \{+1, -1\}$

$$Likelihood(a, X^\ell) = \prod_{i = 1}^{\ell} a(x_i,\theta)^{[y_i = +1]} (1 - a(x_i, \theta))^{[y_i = -1]} → \operatorname*{max}_{\theta}$$

Прологарифмируем правдоподобие выборки и перейдем к задаче минимизации:

$$Q(a, X^\ell) =     -\sum_{i = 1}^{\ell}
        [y_i = +1] \log a(x_i, \theta)
        +
        [y_i = -1] \log (1 - a(x_i, \theta)) \to \operatorname*{min}_{\theta}$$
        
Подставим $a(x, \theta)$ в функцинал качества:

$$ Q(a, X^\ell) = -\sum_{i = 1}^{\ell} \left(
    [y_i = +1]
    \log \frac{1}{1 + \exp(-\langle \theta, x_i \rangle)}
    +
    [y_i = -1]
    \log \frac{\exp(-\langle \theta, x_i \rangle)}{1 + \exp(-\langle \theta, x_i \rangle)}
\right)
=\\
=
-\sum_{i = 1}^{\ell} \left(
    [y_i = +1]
    \log \frac{1}{1 + \exp(-\langle \theta, x_i \rangle)}
    +
    [y_i = -1]
    \log \frac{1}{1 + \exp(\langle \theta, x_i \rangle)}
\right)
=\\
=
\sum_{i = 1}^{\ell}
    \log \left(
        1 + \exp(-y_i \langle \theta, x_i \rangle)
    \right) $$
    

Итоговый оптимизируемый функционал качества (logloss), записанный для меток классов $y \in \{+1, -1\}$ и усредненный по выборке

$$Q(a, X^\ell) = \frac{1}{\ell}\sum_{i = 1}^{\ell}
    \log \left(
        1 + \exp(-y_i \langle \theta, x_i \rangle)
    \right) \to \operatorname*{min}_{\theta}$$

Реализуем его в функции logloss:

In [26]:
def logloss(theta, X, y):
    l=X.shape[0]
    y_temp=np.where(y==0,-1,1)
    result=(np.log(1 +np.exp(-y_temp*np.dot(X,theta)))).sum()*1/l
    return result

In [28]:
assert logloss(theta, X, y).round(3) == 0.861, 'Функция считается неверно'

__Алгоритм оптимизации функционала качества. Стохастический градиентный спуск__

<b>Вход: </b> Выборка $X^\ell$, темп обучения $h$

<b>Выход: </b> оптимальный вектор весов $\theta$

1.  Инициализировать веса $\theta$
2.  Инициализировать оценку функционала качества: $Q(a, X^\ell)$
3.  <b>Повторять</b>:

    Выбрать случайным образом подвыборку объектов $X^{batch} =\{x_1, \dots,x_n \}$ из $X^{\ell}$
    
    Рассчитать градиент функционала качества: $\nabla Q(X^{batch}, \theta)$
    
    Обновить веса: $\theta := \theta - h\cdot \nabla Q(X^{batch}, \theta)$
       
    <b>Пока</b> значение $Q$ и/или веса $\theta$ не сойдутся   

Реализуем функцию рассчета градиента функционала качества

$$\frac{\partial Q(a, X^{batch}) }{\partial \theta_j}   = \frac{\partial \frac{1}{n}\sum_{i = 1}^{n}
    \log \left(
        1 + \exp(- y_i \langle \theta, x_i \rangle)
    \right)} {\partial \theta_j}  = \frac{1}{n}\sum_{i = 1}^{n}
     \frac {1}{
        1 + \exp(- y_i \langle \theta, x_i \rangle)} \cdot  \exp(- y_i \langle \theta, x_i \rangle) \cdot -y_i x_{ij}$$

Реализуйте рассчет градиента в матричном виде:

In [30]:
def gradient(theta, X, y):
    y_pred=binary_class_prediction(theta, X)
    n=X.shape[0]
    result=np.dot(X.T, (y_pred - y)) / n
    return result

assert gradient(theta, X, y).shape == theta.shape, 'Неверный размер массива'

Функция обучения уже реализована

In [31]:
def fit(X, y, batch_size=10, h=0.05,  iters=100, plot=True):

    # получаем размерности матрицы
    size, dim = X.shape

    # случайная начальная инициализация
    theta = np.random.uniform(size=dim)

    errors = []

    theta_history = theta
    colors = [plt.get_cmap('gist_rainbow')(i) for i in np.linspace(0,1,dim)]

    # plt
    if plot:
        fig = plt.figure(figsize=(15, 10))
        ax1 = fig.add_subplot(221)
        ax2 = fig.add_subplot(222)
        ax3 = fig.add_subplot(212)
        fig.suptitle('Gradient descent')


    for _ in range(iters):

        # берём случайный набор элементов
        batch = np.random.choice(size, batch_size, replace=False)
        X_batch = X[batch]
        y_batch = y[batch]

        # считаем производные
        grad = gradient(theta, X_batch, y_batch)

        assert type(grad) == np.ndarray, 'неверный тип'
        assert len(grad.shape) == 1, 'Необходимо вернуть одномерный вектор'
        assert grad.shape[0] == len(theta), 'длина вектора должна быть равной количеству весов'


        # Обновляем веса

        theta -= grad * h

        theta_history = np.vstack((theta_history, theta))

        # error
        loss = logloss(theta, X, y)
        errors.append(loss)

        if plot:
            ax1.clear()
            ax1.scatter(range(dim), theta, label='Gradient solution')
            ax1.legend(loc="upper left")
            ax1.set_title('theta')
            ax1.set_ylabel(r'$\bar \beta$')
            ax1.set_xlabel('weight ID')


            ax2.plot(range(_+1), errors, 'g-')
            ax2.set_title('logloss')
            ax2.set_xlabel('itarations')

            ax3.plot(theta_history)
            ax3.set_title('update theta')
            ax3.set_ylabel('value')
            ax3.set_xlabel('itarations')
            time.sleep(0.05)
            fig.canvas.draw()

    return theta

In [32]:
X, y = make_classification(n_samples=2000)

In [33]:
optimal_theta = fit(X, y)

<IPython.core.display.Javascript object>

In [34]:
y_pred = binary_class_prediction(optimal_theta, X)

Вывод: для обучения моделей большую роль играет предообработка данных: нужно заполнить пропуски в данных(если этих пропусков не слишком много, иначе удалить колонку), заменить выбросы на адекатные значения, преобразовать строковые данные в численные, обработать ситуацию с высокой корелляцией признаков, стандартизировать признаки, при необходимости применить алгоритм PCA. Далее идет этап обучения моделей и подсчет точности пресказаний на тестовой выборке. Для настройки гипермараметров можно применить Жадный поиск, посчитать различные методы качества в зависимости от поставленной задачи, выбрать наилучшую модель